# Installing necessary libraries and packages


In [1]:
!pip install ipywidgets opencv-python opencv-python-headless numpy requests lxml requests beautifulsoup4 lxml ultralytics gdown pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.5 MB/s eta 0:00:00


# Importing them

In [2]:
import ipywidgets as widgets
from IPython.display import display, Image
import numpy as np
from io import BytesIO
import requests
from bs4 import BeautifulSoup
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import time
from lxml import html
import time
from ultralytics import YOLO
import gdown
from PIL import Image

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Preparing & saving model
We have used YOLO v11 for our task. We fine-tuned it on coco dataset to further enhance the performance and saved the model to Google Drive. We will be accessing the model from Drive.

In [3]:
''' Don't uncomment this block '''
# model = YOLO("yolo11n.pt")
# train_results = model.train(
#     data = "coco8.yaml",
#     epochs = 100,
#     imgsz = 640,
#     device = "cpu",
# )
# path = model.export(format = "onnx")

" Don't uncomment this block "

# Loading it back

In [4]:
file_id = '1dXbXe7SsQKyK2N4XKJmJtdDEY0IfXELh'
download_link = f'https://drive.google.com/uc?id={file_id}'
output = 'model.onnx'
gdown.download(download_link, output, quiet = False)
model = YOLO('model.onnx')

Downloading...
From: https://drive.google.com/uc?id=1dXbXe7SsQKyK2N4XKJmJtdDEY0IfXELh
To: /content/model.onnx
100%|██████████| 10.7M/10.7M [00:00<00:00, 101MB/s]

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


# Approach
* First of all, we have created two dictionaries, one for locations and another one for places within locations. For locations, keys are locations and values are their URLs. For places, keys are again locations but values are places within locations.
* We have created 2 user friendly dropdowns. First dropdown is for selecting location and second is to select places within that location.
* Once we have our final place of interest, we extract 'alt' which is a unique identifier for each place. This 'alt' gives us URL of the image we are interested in.
* Once we have URL of image, we directly feed this URL to model and it gives us the image with bounding box.
* Depending on location, it was observed that for some locations images update every minute while for some locations it update every 3 minutes.
* To take care of this inconsistent update frequency we define two variables here 'current_image_url' and 'previous_image_url'.
* For any place, we check for URL of image every minute. If current_image_url is different from previous_image_url, it means image has updated and we also update the same. However, if current_image_url is same as previous_image_url, we simply skip.





In [5]:
locations = {
    'Woodlands & Tuas Checkpoint': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/woodlands.html#trafficCameras',
    'Kranji Expressway (KJE)': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/kje.html#trafficCameras',
    'Seletar Expressway (SLE)': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/sle.html#trafficCameras',
    'Bukit Timah Expressway (BKE)': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/bke.html#trafficCameras',
    'Ayer Rajah Expressway (AYE)': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/aye.html#trafficCameras',
    'Sentosa Gateway': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/stg.html#trafficCameras',
    'Central Expressway (CTE)': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/cte.html#trafficCameras',
    'Marina Coastal Expressway (MCE)': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/mce.html#trafficCameras',
    'Tampines Expressway (TPE)': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/tpe.html#trafficCameras',
    'Kallang-Paya Lebar Expressway (KPE)': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/kpe.html#trafficCameras',
    'East Coast Parkway (ECP)': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/ecp.html#trafficCameras',
    'Loyang Ave/ Tanah Merah Coast Road': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/ltm.html#trafficCameras',
    'Pan-Island Expressway (PIE)': 'https://onemotoring.lta.gov.sg/content/onemotoring/home/driving/traffic_information/traffic-cameras/pie.html#trafficCameras'
}


places = {
    'Woodlands & Tuas Checkpoint': ['View from Second Link at Tuas', 'View from Tuas Checkpoint', 'View from Woodlands Causeway (Towards Johor)', 'View from Woodlands Checkpoint (Towards BKE)'],
    'Kranji Expressway (KJE)': ['View from Choa Chu Kang West Flyover', 'View from Entrance From Choa Chu Kang Dr', 'View from Exit To BKE', 'View from Tengah Flyover'],
    'Seletar Expressway (SLE)': ['View from Lentor Flyover', 'View from Mandai Flyover', 'View from Marsiling Flyover', 'View from SLE(BKE) Exit', 'View from Ulu Sembawang Flyover', 'View from Upp Thomson Flyover'],
    'Bukit Timah Expressway (BKE)': ['View from after KJE Exit', 'View from Chantek Flyover', 'View from Dairy Farm Flyover', 'View from Exit 5 to KJE (Towards Checkpoint)', 'View from Mandai Rd Entrance', 'View from Woodlands Flyover (Towards Checkpoint)'],
    'Ayer Rajah Expressway (AYE)': ['View from After Tuas West Road', 'View from Clementi Ave 6 Entrance', 'View from Entrance from Benoi Rd', 'View from Entrance from Jln Ahmad Ibrahim', 'View from Entrance from Yuan Ching Rd', 'View from Keppel Viaduct', 'View from Lower Delta Road', 'View from Near Dover Drive', 'View from Near NUS', 'View from Near West Coast Walk', 'View from Towards Alexandra Road', 'View from Towards Pandan Gardens'],
    'Sentosa Gateway': ['View from Sentosa Gateway (Towards Sentosa)', 'View from Sentosa Gateway (Towards Telok Blangah)'],
    'Central Expressway (CTE)': ['View from Ang Mo Kio Ave 1 Flyover', 'View from Ang Mo Kio Ave 5 Flyover', 'View from Braddell Flyover', 'View from Bukit Merah Flyover', 'View from Entrance from Chin Swee Road', 'View from Exit 6 to Bukit Timah Road', 'View from Moulmein Flyover', 'View from St George Road', 'View from Yio Chu Kang Flyover'],
    'Marina Coastal Expressway (MCE)': ['View from Marina Boulevard', 'View from Marina Boulevard/Marina Coastal Drive', 'View from Maxwell Road', 'View from MCE 1.02km', 'View from MCE/ECP'],
    'Tampines Expressway (TPE)': ['View from Entrance To Tampines Flyover', 'View from Exit to Punggol Flyover', 'View from Seletar Flyover', 'View from Seletar West Link', 'View from Tampines Ave 10 Entrance', 'View from TPE(KPE) Exit', 'View from Upp Changi Flyover Towards PIE'],
    'Kallang-Paya Lebar Expressway (KPE)': ['View from Defu Flyover', 'View from Kallang Bahru', 'View from Kallang Way Flyover', 'View from KPE/ECP', 'View from KPE/PIE', 'View from Tampines Flyover'],
    'East Coast Parkway (ECP)': ['View from Benjamin Sheares Bridge', 'View from Entrance from MCE', 'View from Entrance from PIE', 'View from Exit 2A to Changi Coast Road', 'View from Laguna Flyover', 'View from Marine Parade Flyover', 'View from Tanjong Katong Flyover', 'View from Tanjong Rhu'],
    'Loyang Ave/ Tanah Merah Coast Road': ['View from Tanah Merah Coast Road towards Changi', 'View from TPE(PIE), Exit 2 to Loyang Ave', 'View from TPE(PIE), Tampines Viaduct'],
    'Pan-Island Expressway (PIE)': ['View from Adam Road', 'View from Bedok North', 'View from Bukit Timah Expressway', 'View from Entrance from Jalan Anak Bukit', 'View from Entrance from Simei Ave', 'View from Entrance to PIE from ECP Changi', 'View from Eunos Flyover', 'View from Exit 27 to Clementi Ave 6', 'View from Exit 35 to KJE', 'View from Hong Kah Flyover', 'View from Kallang Way', 'View from Kim Keat Link', 'View from Mount Pleasant', 'View from Nanyang Flyover', 'View from Paya Lebar Flyover', 'View from Thomson Flyover', 'View from Tuas Flyover', 'View from Woodsville Flyover']
}


location_dropdown = widgets.Dropdown(
    options = locations.keys(),
    value = None,
    description = 'Select a Location',
    style = {'description_width': '150px'}
)


place_dropdown = widgets.Dropdown(
    options = [],
    value = None,
    description = 'Select a Place',
    style = {'description_width': '150px'}
)


fetch_places_button = widgets.Button(description = 'Fetch Places')
download_image_button = widgets.Button(description = 'Monitor Traffic')


def fetch_places(button):
  location = location_dropdown.value
  if location in places:
    place = places[location]
    place_dropdown.options = place


def fetch_image(button):
  selected_location = location_dropdown.value
  selected_place = place_dropdown.value
  page_url = locations[selected_location]
  previous_image_url = None
  counter1 = {'motorcycle': 0, 'car': 0, 'bus': 0, 'truck': 0}
  for i in range(30):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'lxml')
    image = soup.find_all('img', alt = selected_place)
    if image:
      current_image_url = 'https:' + image[0]['src']
      if current_image_url != previous_image_url:
        response = requests.get(current_image_url)
        results = model(current_image_url)
        vehicle_classes = {3: 'motorcycle', 2: 'car', 5: 'bus', 7: 'truck'}
        counter2 = {'motorcycle': 0, 'car': 0, 'bus': 0, 'truck': 0}
        for result in results:
          for box in result.boxes:
            cls_id = int(box.cls[0])
            if cls_id in vehicle_classes:
              vehicle_type = vehicle_classes[cls_id]
              counter1[vehicle_type] += 1
              counter2[vehicle_type] += 1
        print('\n\n\n')
        print(f'Image {i + 1}/30')
        print('Total vehicle count in this image:')
        for vehicle, count in counter2.items():
          print(f"{vehicle}: {count}")
        print('Total vehicle count till now:')
        for vehicle, count in counter1.items():
          print(f"{vehicle}: {count}")
        results[0].show()
        print('\n\n\n')
      else:
        print(f'No change in image URL, skipping image {i + 1}.')
    else:
      print('Image not found, please check the camera availability.')
    previous_image_url = current_image_url
    time.sleep(60)


fetch_places_button.on_click(fetch_places)
download_image_button.on_click(fetch_image)

# Inference
* Kindly run the cell below and choose location and place from dropdown. Once you hit on 'Monitor Traffic' button, it will go live and will start showing live image with bounding box around vehicles.
* An individual and a cummulative counter has also been added which will show total vehicle count in current image and total vehicle count till now.
* It will update on its own every minute for 30 minutes. If there is an update on site, it will show here. If not, it will skip it.





In [6]:
display(location_dropdown, fetch_places_button, place_dropdown, download_image_button)

Output hidden; open in https://colab.research.google.com to view.